# Tutorial for the new data normalisation

In [1]:
import processing as pr

In [2]:
import os
import logging
import importlib

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import extra_data as ed
import extra_geom as eg

import toolbox_scs as tb
import toolbox_scs.detectors as tbdet

In [3]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 12)

In [4]:
xr.set_options(display_style='text')

In [5]:
proposal = 2530
dark_run = 49
run = 50

pattern = ['image', 'dark'] * 99 + ['end_image']

To keep the runtimes of this notebook short we only use the first twenty trains.

In [6]:
train_indices = range(20)

## Step 1

In the first step we create an averaged dark image and corrected images. The calculations are done per module. The normalisation is done as follows:

- Ri is a run with actual images (X-ray pulses) with the sequence (Ri_image, Ri_intradark, Ri_image, ...)
- Rd is a dark run with sequence (Rd_dark, Rd_intradark, Rd_dark, ...)

The normalisation per module is done as:

average{ [Ri_image - average(Ri_indradark) - (average(Rd_dark) - average(Rd_intradark)]) / XGM}

Averaging is done both over trains and pulses. Only trains with valid XGM data are used.

During the actual analysis this normalisation is done with scripts.

### create dark images from the dark run for all modules

This part calculates the averaged dark image: [average(Rd_dark) - average(Rd_intradark)]

In [7]:
for module_number in range(16):
    module_dark = pr.Module(proposal=proposal, run=dark_run,
                            module=module_number, pattern=pattern)
    module_dark.process_std(train_indices=train_indices,
                            dirname='processed_runs_xgm')

### create normalised images for all modules
Now the full normalisation is calculated.

In [8]:
for module_number in range(16):
    module = pr.Module(proposal=proposal, run=run,
                       module=module_number, pattern=pattern)
    module.process_normalised(dark_run=dark_run,
                              train_indices=train_indices,
                              dirname='processed_runs_xgm')

## Step 3: collect per-module images and put them together to one image
After calculating normalised images for all modules we need to group the individual files into one xarray. 

In [9]:
images = pr.concat_module_images(dirname='processed_runs_xgm', run=run)

KeyError: "Unable to open object (object 'data' doesn't exist)"

Now the analysis can be continued as done previously.

`images` obtained in the previous cell equals `images` in ``shared/tutorial_analysis.ipynb`` from where the rest of this notebook is taken.
## Step 4: start with the actal analysis
#### copied from ``shared/tutorial_analysis.ipynb``

In [10]:
#quad_pos = [(-124.100, 3.112),  # TR
#            (-133.068, -110.604),  # BR
#            (0.988, -125.236),  # BL
#            (4.528, -4.912)  # TL
#            ]
#

quad_pos = [(-123.0 , 1.72),  # TR
            (-130.0, -110.18),  # BR
            (1.4, -122.78),  # BL
            (2.2, -4.08)  # TL
            ]

geopath = '/gpfs/exfel/sw/software/git/EXtra-geom/' \
                  'docs/dssc_geo_june19.h5'
geom = eg.DSSC_1MGeometry.from_h5_file_and_quad_positions(geopath, quad_pos)

#geom = tbdet.load_geom()
#geom.inspect()

In [ ]:
#Some ASICS (squares) are broken and can be manually masked away if desired

masklist = [
    (4, 1, 0),
    (13, 0, 0),
    (1, 0, 6),
]
dssc_mask = tbdet.quickmask_DSSC_ASIC(geom, masklist)

In [ ]:
#Assemble the modules to one image. This has not yet been corrected hexagonal to carthesian
image, center = geom.position_modules_fast(images)


In [ ]:
plt.imshow(image,vmin=-0, vmax=2, aspect=204.0/236.0)
plt.colorbar()